# Data Analysis Workshop

## Tutorial III: Accessing Data on S3

<div style="
  border-left: 6px solid rgb(236, 114, 0);
  background-color:rgb(253, 231, 157);
  color:rgb(19, 19, 18);
  padding: 1em;
  font-size: 110%;
  border-radius: 4px;
  margin: 1em 0;
">
⚠️ <strong>ATTENTION</strong>: if the kernel is not already set, please change it to the <code>DA Workshop (python)</code>
</div>

This tutorial is similar to the first one! However we’ll learn how to use `xarray`, `zarr` and `s3fs` to explore datasets available via S3 but also upload results to it!

To get started, we’ll run a simple analysis on a remote NCAR dataset (hosted on AWS S3) to plot global monthly means of near-surface temperature, using data from the [CESM2 Large Ensemble (LENS2)](https://www.cesm.ucar.edu/community-projects/lens2).

LENS2 is a 100-member CESM2 ensemble (1850–2100) based on CMIP6 historical and SSP370 forcing. It includes two 50-member subsets: one using original biomass burning emissions, and another using a smoothed version (SMBB) to reduce discontinuities.

In [ ]:
from freva_client import databrowser

In [ ]:
databrowser.metadata_search(fs_type="s3")

In [ ]:
search_keys = {
    'fs_type': 's3',
    'project':'cesm2-le',
    'experiment': 'historical',
    'realm': 'atm',
    'time_frequency':'monthly'
}
'ts' in databrowser.metadata_search(**search_keys)['variable']

In [ ]:
db = databrowser(variable='ts', **search_keys)
list(db)

## Let's quickly check how to open the first dataset with xarray

Since all are zarr datasets we can use `xr.open_zarr` or `xr.open_dataset(engine='zarr')`.

Because the data is public, we do not need credentials to open the data. We specify `anon` (anonymous) in order for xarray to not sign the requests.

If data is **not** public we will get `403 Forbidden`

In [ ]:
import xarray as xr
xr.open_zarr(list(db)[0], storage_options={ 'anon':True })
xr.open_dataset(list(db)[0], engine='zarr', storage_options={ 'anon':True })

In [ ]:
import xarray as xr
import numpy as np
def field_mean(
    data: xr.DataArray,
    lat_name: str = "lat",
    lon_name: str = "lon",
    mean_dims: tuple[str, str] = ("lat", "lon")
) -> xr.DataArray:
    """
    Compute an area-weighted mean over latitude and longitude using cosine latitude weights.

    Parameters
    ----------
    data : xr.DataArray
        Input data array with latitude and longitude coordinates.
    lat_name : str, optional
        Name of the latitude coordinate. Default is "lat".
    lon_name : str, optional
        Name of the longitude coordinate. Default is "lon".
    mean_dims : Sequence[str], optional
        Dimensions over which to compute the mean. Default is ("lat", "lon").

    Returns
    -------
    xr.DataArray
        Area-weighted mean of the input data over the specified dimensions.
    """
    # Extract latitude values
    lat = data[lat_name]

    # Compute cosine of latitude in radians
    weights = np.cos(np.deg2rad(lat))

    # Normalize weights so they sum to 1 (over just lat)
    weights /= weights.sum()

    # Apply weighted mean over specified dimensions
    return data.weighted(weights).mean(dim=mean_dims, keep_attrs=True)

In [ ]:
from pathlib import Path
s3_opts = { 'anon':True }
time_series = {}

for fileurl in db:
    print(f"Opening {fileurl}...")
    engine = {'engine':'zarr'} if fileurl.endswith('zarr') else {}
    ds = xr.open_dataset(fileurl, **engine, storage_options=s3_opts)
    ds = ds.rename({'member_id':'ensemble'})
    
    # Going through all ensembles might take some time
    # let's make a cut at 5 member for demo purposes
    # Memory usage might spike to ~70GB 
    members=ds.ensemble[:5]
    ds = ds.sel(ensemble=members)    
    
    dataset_name=Path(fileurl).stem   
    
    mean_ts = field_mean(ds["TS"])
    mean_ts.attrs['source_dataset'] = fileurl
    time_series[dataset_name] = mean_ts
   

In [ ]:
import matplotlib.pyplot as plt

# Create a color cycle for different experiments
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

plot = plt.figure(figsize=(12, 5))

for i, (exp, data) in enumerate(time_series.items()):
    ts = data.resample(time="1YE").mean()
    ts_min = ts.min(dim="ensemble").squeeze()
    ts_max = ts.max(dim="ensemble").squeeze()
    ts_mean = ts.mean(dim="ensemble").squeeze()
    time_str = ts.time.dt.strftime("%Y").values
    time_values = np.array(ts.time.values, dtype='datetime64[ns]')
    # Plot min–max shading
    plt.fill_between(
        time_values,
        ts_min.values,
        ts_max.values,
        color=colors[i % len(colors)],
        alpha=0.2,
        label=None,
    )
    # Plot mean line
    plt.plot(
        time_values,
        ts_mean.values,
        color=colors[i % len(colors)],
        linewidth=2,
        label=f"{exp}"
    )

# Add plot decorations
plt.title("Ensemble Mean and Spread for Each Dataset")
plt.xlabel("Time")
plt.ylabel(f"{data.attrs['long_name']} [{data.attrs['units']}]")  # Replace with actual units
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

----

# Let's save the plot on S3, as well as the datasets!

We start by setting up a dictionary with the configuration we will need.

In [ ]:
from getpass import getuser # to get USER env variable
USERNAME = getuser()
s3_config = {
    'bucket' : 'freva',
    'endpoint' :'https://s3.eu-dkrz-1.dkrz.cloud', # DKRZ Minio S3
    'prefix' : f'workshop/{USERNAME}', # Avoid users writting object with same prefix
    'access_key_id' : "s3handson", # Only valid during the workshop
    'secret_access_key' : "s3handson", # Only valid during the workshop
    'region' : 'eu-dkrz-1',
}

Now we create a wrapper on S3 that mimics a local filesystem. This will be important for `netcdf`

In [ ]:
import s3fs
s3 = s3fs.S3FileSystem(
    key = s3_config['access_key_id'],
    secret = s3_config['secret_access_key'],
    endpoint_url = s3_config['endpoint'],
)

full_prefix = s3_config['bucket']+'/'+s3_config['prefix'] 
testobj = f'{full_prefix}/hi.txt'
s3.write_bytes(testobj, b'Hi!\n')

print(f"Writing data to S3 works! Test it with:\ncurl {s3_config['endpoint']}/{testobj}")

In [ ]:
!curl https://s3.eu-dkrz-1.dkrz.cloud/freva/workshop/${USER}/hi.txt

### Save the figure on S3
 - Open the object ("file") on S3 where the figure should be saved
 - Save the figure into that opened object!

In [ ]:
figure_path = f'{full_prefix}/figure-ts-mean.png'
with s3.open(figure_path, 'wb') as f:
    plot.savefig(f)
print(f"Open this link to get the image: {s3_config['endpoint']}/{figure_path}")

### Let's now write the data to S3 as both NetCDF and ZARR

Note that since we are writing to an object store, we cannot use `xr.to_netcdf` with an open file object — `seek` operations are not supported in this context!

Instead, simply save the file locally first, then copy it to the S3 bucket.

In [ ]:
def write_netcdf(s3_path, dataset:xr.Dataset):
    tmp_name=f"/scratch/{USERNAME[0]}/{USERNAME}/{name}.nc"
    dataset.to_netcdf(tmp_name, engine='h5netcdf')
        
    ## Copy to s3
    with s3.open(s3_path,'wb') as s3file:
        with open(tmp_name, 'rb') as tmpf:
            s3file.write(tmpf.read())

    ## Check if we can open with Xarray!
    with s3.open(s3_path,'rb') as s3file:
        xr.open_dataset(s3file, engine='h5netcdf')
    
    ## We can now remove the local copy
    import os
    os.remove(tmp_name)

In [ ]:
def write_zarr(store, dataset: xr.Dataset):     
    import zarr
    zarr.config.set(default_zarr_format=2)
    
    import numcodecs
    codec = numcodecs.Blosc(shuffle=1, clevel=6)
    
    data_encoding = {}
    for var in dataset.variables:
        data_encoding[var] = { "compressors": codec}

    dataset.to_zarr(store,
                    mode='w', # OVERWRITES existing data!!
                    encoding=data_encoding,
                    consolidated=True, # consolidate metadata for fast access
                    storage_options= { # We cannot use the anonymous mode anymore
                        'key':s3_config['access_key_id'], 
                        'secret':s3_config['secret_access_key'],
                        'endpoint_url': s3_config['endpoint'],
                    })

In [ ]:
def write_dataset_to_s3(name:str, dataset:xr.Dataset, file_format='nc'):
    _supported_types = ('nc','zarr')
    file_format = file_format.replace('.','')
    if file_format not in _supported_types:
        raise Exception('Unsuported file format, use one of')
    
    s3_path = f'{full_prefix}/{name}.{file_format}'
    if file_format == 'nc':
        write_netcdf(s3_path, dataset)

    elif file_format == 'zarr':
        write_zarr(f's3://{s3_path}', dataset)

    netcdf_mode= 'zarr' if file_format == 'zarr' else 'bytes'
    print(f"Try running:\n\t" \
          f"/fastdata/freva/opt/bin/ncdump -h {s3_config['endpoint']}/{s3_path}#mode=s3,{netcdf_mode}\n\t"\
          f"/fastdata/freva/opt/bin/cdo sinfo {s3_config['endpoint']}/{s3_path}#mode=s3,{netcdf_mode}")
    

We save the mean temperature array in S3 as both Zarr and NetCDF

In [ ]:
for name, data in time_series.items():
    ## for CDO it is important that time is the first dimension
    ## also it cannot have indexers of type string
    ensemble = data.ensemble.values
    ensemble_id = np.arange(len(ensemble))
    
    dataset = data.to_dataset() \
            .transpose('time',...) \
            .assign_coords(ensemble_id=('ensemble',ensemble_id)) \
            .swap_dims({'ensemble':'ensemble_id'}) \
            .reset_coords(drop=True)\
            .assign_attrs(ensembles= ', '.join(data.ensemble.values))
    
    write_dataset_to_s3(name, dataset, file_format='nc')
    write_dataset_to_s3(name, dataset, file_format='zarr')

public_url = f"https://eu-dkrz-1.dkrz.cloud/browser/{s3_config['bucket']}/{s3_config['prefix']}"
f"{s3_config['endpoint']}/browser/{s3_config['bucket']}/{s3_config['prefix']}"
print(f"Browser the data in {public_url}")

A bespoke compiled binary of cdo and ncdump will allow us to check the content of these S3 files:

1. for NetCDF:

In [ ]:
!/fastdata/freva/opt/bin/ncdump -h https://s3.eu-dkrz-1.dkrz.cloud/freva/workshop/$USER/cesm2LE-historical-smbb-TS.nc#mode=s3,bytes
!/fastdata/freva/opt/bin/cdo sinfo https://s3.eu-dkrz-1.dkrz.cloud/freva/workshop/$USER/cesm2LE-historical-smbb-TS.nc#mode=s3,bytes

2. for Zarr:

In [ ]:
!/fastdata/freva/opt/bin/ncdump -h https://s3.eu-dkrz-1.dkrz.cloud/freva/workshop/$USER/cesm2LE-historical-smbb-TS.zarr#mode=s3,zarr
!/fastdata/freva/opt/bin/cdo sinfo https://s3.eu-dkrz-1.dkrz.cloud/freva/workshop/$USER/cesm2LE-historical-smbb-TS.zarr#mode=s3,zarr

We can also test whether the s3 files can be opened with xarray:

In [ ]:
print("Opening")
for name in time_series.keys():
    file = f'https://s3.eu-dkrz-1.dkrz.cloud/freva/workshop/{USERNAME}/{name}'
    print(f"{file}.nc\n{file}.zarr")
    xr.open_dataset(f'{file}.nc', engine='h5netcdf')
    xr.open_zarr(f'{file}.zarr')

print('All output datasets opened!')